Dan Shea  
2021-05-22  
#### Using Monte Carlo to estimate $\pi$ revisited
I coded this up about a decade ago in `wxpython` with a gui to illustrate the application of Monte Carlo method to approximate the value $\pi$.  
However, I don't think it runs anymore (as `wxpython` has changed quite a bit in the intervening years).  
So Let's run through it again and this time we'll use `ipycanvas` to illustrate the method.  
First, let's briefly walk through the experimental setup.  

#### Experimental setup
If we inscribe a unit circle inside of a unit square and begin to rnadomly throw darts at the defined area, the number of darts that land inside of the circle should roughly be in proportion to the area of the circle to the square. And the more darts we throw, the better our approximation would become.

To that end, we can construct the formula as follows:

$$\frac{P_{c}}{P_{s}} = \frac{A_{c}}{A_{s}} = \frac{\pi \cdot r^{2}}{4 \cdot r^{2}} = \frac{\pi}{4}$$

And we know that all of the darts would land inside the square, so that is equal to the total number of darts thrown.  
Therefore, we may re-write this to approximate $\pi$ as follows:

$$\pi \approx 4 \cdot \frac{P_{c}}{P_{total}}$$

Since we are on a computer, we'll substitue our darts for points randomly selected from a uniform distribution spanning the coordinates of our square and see what proportion of them land inside the circle in relation to the total number of points drawn. Multiply that by 4 and see if our answer is close to the value of $\pi$.

In [1]:
from ipycanvas import MultiCanvas, hold_canvas
import numpy as np
from time import sleep

In [2]:
# Function to draw random (x,y) coordinates of our points
def draw_points(size=10000):
    return np.random.randint(200, size=(size,2))

# function to see if a point lies within the circle
def isCircle(coords, center=(99,99), radius=100):
    '''
    coords is a tuple of (x,y) coordinates of the point
    center is a tuple of (x,y) coordinates of the circle center
    radius is the radius of the circle
    '''
    # length of the legs of a right triangle formed by the point and the circle center
    a = abs(coords[0] - center[0])
    b = abs(coords[1] - center[1])
    # If the hypotenuse of that right triangle is shorter than the radius of the circle, we are inside the circle
    return np.sqrt(a**2 + b**2) < radius

In [3]:
points = draw_points()
# We set the size of the square to be 200 pixels on a side in our draw points function, so the circle center is at (99,99) 0-based indices
# and the radius is a length of 100
results = np.apply_along_axis(isCircle, 1, points)
num_circle = results.sum()
4 * (num_circle / len(results))

3.1556

#### Our approximation of $\pi$
Above, we can see that our approximation using 10,000 points is not too bad.  
Let's use `ipycanvas` to illustrate our experiment and show how the approximation changes as we add more points to the simulation.
I will use the same points drawn earlier, but we will step through one point at a time and update out approximation of $\pi$ as we go.

In [4]:
canvas = MultiCanvas(2, width=800,height=200)
canvas[0].line_width = 1.0
# Draw the bounding box
canvas[0].stroke_style = 'black'
canvas[0].stroke_rect(0, 0, 200, 200)
# Draw the inscribed circle
canvas[0].stroke_style = 'red'
canvas[0].stroke_circle(99,99,100)
canvas

MultiCanvas(height=200, width=800)

In [5]:
# Place the initial estimate for pi on the canvas
canvas[1].fill_style = 'black'
canvas[1].font = '18px courier'
pi_estimate = 0.0
index = 0
canvas[1].fill_text(f'estimate: {pi_estimate}', 210, 20)
canvas[1].fill_text(f'  points: {index}', 210, 40)

In [6]:
for result in results:
    with hold_canvas(canvas):
        if result == True:
            canvas[0].fill_style = 'red'
        else:
            canvas[0].fill_style = 'black'
        canvas[0].fill_rect(points[index][0], points[index][1], 1)
        tmp_points_in = results[0:index+1].sum()
        pi_estimate = 4 * (tmp_points_in / (index+1))
        canvas[1].clear()
        canvas[1].fill_text(f'estimate: {pi_estimate}', 210, 20)
        index += 1
        canvas[1].fill_text(f'  points: {index}', 210, 40)
        sleep(0.01)